In [1]:
import pandas as pd
import pickle
import json
import os
import google.oauth2.credentials
import googleapiclient.discovery

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

 
# The CLIENT_SECRETS_FILE variable specifies the name of a file that contains
# the OAuth 2.0 information for this application, including its client_id and
# client_secret.
CLIENT_SECRETS_FILE = "client_secret.json"
 
# This OAuth 2.0 access scope allows for full read/write access to the
# authenticated user's account and requires requests to use an SSL connection.
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

In [2]:
def get_authenticated_service():
    credentials = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)
    #  Check if the credentials are invalid or do not exist
    if not credentials or not credentials.valid:
        # Check if the credentials have expired
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CLIENT_SECRETS_FILE, SCOPES)
            credentials = flow.run_console()
 
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)
 
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [3]:
def write_csv(comments):
    df = pd.DataFrame(comments)
    df.columns = ["Comments"]
    print(df)
    df.to_csv("Comments.csv", encoding='utf-8', index=False)

In [4]:
def get_video_id(service, url,limit):
    id = url.split("v=")[1]
    video_id = id.split("&")[0]
    
    get_video_comments(service,limit, part="snippet",
                                maxResults=100,
                                order="time",
                                textFormat="plainText",
                                videoId=video_id)

In [5]:
def get_video_comments(service, limit, **kwargs):
    comments = set()
    
    request = service.commentThreads().list(**kwargs)
    response = request.execute()
    try:
        while(response["nextPageToken"]):
            print(response["nextPageToken"])

            for item in response['items']:
                comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
                comments.add(comment)
                if(len(comments)>=limit):
                    return write_csv(comments)
            try:
                request = service.commentThreads().list(**kwargs,pageToken=response["nextPageToken"])
                response = request.execute()
            except HttpError:
                os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
                service = get_authenticated_service()
            
    except KeyError or KeyboardInterrupt:
        return write_csv(comments)

In [6]:
if __name__ == "__main__":
    # When running locally, disable OAuthlib's HTTPs verification. When
    # running in production *do not* leave this option enabled.

    os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
    service = get_authenticated_service()

#     url = input("Enter video link: ")
    url = "https://www.youtube.com/watch?v=uoxtAXN2tnI"
    
    max_comments = int(input("How many comments approximately do you want ? (NUMBER): "))

    get_video_id(service,url,max_comments)

How many comments approximately do you want ? (NUMBER): 110000
QURTSl9pM3VDV1ZJN1FCWFdYdV84RVE4Xy0wcVBYRm8wNS1MS25LNnVPUGVNYndCMVo5VzktSEpkTDVFaGdtU3NBcWY0YUhmRGJBbTBPbmJyYkUxZVlFUnFFbkJqZjZmX0l2ZWRVdTZ2czNfWHFlWEExQkJVM19aNldmc0Vobmg=
QURTSl9pMTFvZDBsR2o4N0FPYkJzWnN5RTNmcXJkdHpTMmZrX2g3QlBlZ3QtVmRDQWJYVDBkTVdiVWFyZ2o4eVJGRzd4LW9HQWFuUkpSWjNBa2JMcm00c1RRMU15Z2tUNnVSZy1iQzVka2cxT0FtYlZ5QU9rcFV6d1d6WG00VVQ0dWxabkpHQXFodjYyVE0=
QURTSl9pMDNGMTdiWFJ4d3Zvc3FoUGpZN3JvdkhmekZ5NzNQaXlNODBDYUtPYlJpc0RTUEdyVjM4em5sVURySFVqdVpSUDI3TWI3emhQSmdLbWtMZnRoTEJ2dmoyNHoxZmZua284aW5iNkdCOUtfVktNVzgzVHE3WnFTNTdLNzlJY0RlRFgtNUpxMUUzRnc=
QURTSl9pM3JFS0YtRFdqOTZPME9BOVR4LWtMbW5mbG5idXpGUmdHb0w4NmRFVU1FeWFKNWZvYnBYRkNsZjJyYVdTMTl0ejR2UGFfTUtKclBCWW1fR0p5M0hEUGZoQ3N1U3I0N0h0VF9sX2dxVUtXXzF2OVZMOE5GSFhjVkx3elRxaTdDVVZfclNpS2N0N0E=
QURTSl9pMmt0LXh3eVlGS2lLaUlnZV85UnBlRERCazkwbU1NV19UT28yV1FfdHNZNWtKUkFHLUFwRWM1QmNVeXdWRFpKVE1CZlZjY0dIMkljTEd0d1lOMnBWMTZhTTBQM2gyLWJVTy10djVTTEZlV3VSR194Qk95MDRnUFRYZkdBSGFwUHRkRUN2b

QURTSl9pM09Yak5BU2lwYW9TZlBxQVozZHVreDlzaHhTRnBnWUZqMV9sT0UzMVA3NlNrcVdWQXN1dEdnTkRmbkZvVUtvaVVmejY2OWZlM1MwVnJHTXpneWxkakNMOHVpQTFsYzlWUzlnbGM4WDlmQlFvTmh0dWxZUExIeTJ4V2E0TDQ0ZXFoSjVWb2cyanM=
QURTSl9pMzZSM09raDFGclRTOXhSdG5NZTVvUzB5alo4R1RRZWdQWk8wNG1GbzFua1JvQ25uMUFZc3hQR0VCdFExTE9aM3RuTFM1N21tWFNwNWR4cXpWSXJhMTJMWEFIZ1B1eHJZSU1ZSDlIczdWdlN6RXk0NEplT0VUckdWX0dTWmRubTZacGFCbTEwbmc=
QURTSl9pMXplbXVEbUducFVFa3RCQ0tlaFgyMnAwLUliQ01NRW1xRkFjUk03djRvZHdvdUJMRVlSZXFBUm14MWdZODhTVC1iUE1FcEhURXRGclJzWjNlMk56dk9uUzRMTGt2dGpreEVUN1Mzc3dBSWpMa0dGWmF5U1BGell2azZPaHRsUjBHaHVGeENYcG8=
QURTSl9pM1RtdHBsVlg5N2ZlSVhmSWlmOU4tNTJSbEp5X3lKd0xVcjU2UHJqZjEwYmlsdlR3eXllWVZ6VXMxeGxEeDFwcnNZdjBVSjdac29UUjFRSEIzS3lEOWNMQ0lxR2RFOWxQT1MyT2VoSkNGSjY4MURIYUZnNGZpdU12NWpnOW5rei1CZU5CYUl6MjQ=
QURTSl9pMHNadUhNOVN4NmhPSjdnZGV0aWtVb1FWLVZtLXhBSUw4c0hPV2FoTnhGVXBndlZDNnRqa2E3Sl9MMnVGay1ZWFZPOEt4Zy1KTTRBb1lJd2JLZjF4WVBIblRhc0RWVENYS2pjTTlheU9ULXJkN01HUmZzcDNzdnMyZDVQNF9Ma2t3QjJsZ2RWd3M=
QURTSl9pMHpRNms4NDZnM3U0UnlEdkVKWFp

QURTSl9pMVRTR0dKZkdBdEhxS3hyV2czWnFZUTljQVl6Qk9MREZvdEhsRHRjRml3eUFRTTZsR1g5OE1UdlptZGo3a3huOUtQcE4xWlpEZnZuSWNFX3RJeWt3TV8wb0dsd1pDem83a2xSRUlOQlVQV2V6RkhMU29ZVHFvbTlDVmYzdkFSTllnOWFTemdVSkU=
QURTSl9pM0NKemJWV2ZzSWVkNHdydHlBbHg5YVRhZXZaQnYwb3lDbkFQcXNwRktfdlFHejdpcE9oNDRUTVJab2VVRE5sV2VxeTJQMThJa2ZLblh2WEt5bVFtVVZONnRyOVJ0WmFrOTRpXy1ZMExHMUxIVThBNGNISnVMVUJfTVJaTDUyamx0SlFpbDdvYkU=
QURTSl9pMGV3NnRtR1dvdzFGMHNtU1J4LVFtbWxJWE9lSXFCYkxWdzB0ek1uSUp5Y3ZMUEFSaTRJekhnUHcwVnFmVkl6Xy1GVGx3NFRNV0hLQ2J2eGphOC11WThVeHBhSEpONldYSW8wbWM5OHJXU1FHRl9jOHVSeG9zMVdOYzAxczB3U1hoekg0a3k2dEU=
QURTSl9pMzgxd29tUGVLWlM4WjV6QXJ1ZGxHdFhOT3BMZHFtZDY5U0lOalZaTWdoeDhJWXVVOXdWV2xRY0hoTnd1WUJPSkdDZE4xLThDSXNjal9PMnI5dzY0djRoeGtOdGRTZUw3Rl9Ibjd6d1RNRGExWU40MmRYSVNyWm5fMW1ZVU5fQ1ktR3hDSjFjMmM=
QURTSl9pMWZRN0RYdkEzUldENGRyVzM4Nkg5Z3hYRzJoc2dfbVQ4UGpvU1owZ3pUSldWT0hwNFlOVG9VV2l2M1MtMC1BTklxUG1kQVhtZFVjVnFoOW9hN2Z6MnJZdFBIeXJNU2ZSRUJyNUpna1ltLTFmUlZVdVc1ZFNyM0doS3ZjeHBqalkxZVc1YmFCRnc=
QURTSl9pMEhVQWlJU05RbVBMT0puV1FqdF8

QURTSl9pMVMwbVBlV0V1V1hFV3J3cUVpZjhRZl8xY0RGNHdIajdrYWZ5RUx5Vkd5LUJjX3VZcmZUcXdUNzVGTHAteFREWG1IM1ZwWjhYRXhMd1JMY2xLbGhRRjVZNkZUM01MWnpLY042OGY5TDJDOGJPUXM1OFI3Y1V3U0d2WTdmUVNtMEgycEFmQWlJR0xS
QURTSl9pMlEtMTJvV0U5TVRWZ3NoVmFWTS0tVUNCU0RzMlhpUGVUWWZ2WGZoQnV3N3BkaTJvMmI3YVQwbmwtQ242OThTX21VcU9FVTBLVy1NRzFXZmxMQll6cXUzV3BRMWVzdTRIdThYTUM5YTgwQjg2NW9rTWhEWFhIazVQN1V4N0MxNUtvWDlxVnJJUlVU
QURTSl9pMDZTYVZwVXhHSDZmek1fWktxbGhTcnc0bUM4UGJXMWNDZjY2cllrdTVtN0hRcjFqVUxYazlGblJHS0ktSU9OYTM2dnd6NklKT2JMRXdDTGR2Sk9OQWk2QmdOeDB3eExJRl84U2oyYVpKVHFYNDNHWEZFMTZHVXNGSm5MZkxsNlNmajczTzVaMHlt
QURTSl9pMEpSMHR3SFMwRkdpQmdJbURWVnUtc3RhdHV2Mkw1a19HUk42aGdwQkZJem53dW93NDJSMzVyQmRYM0lJN0hRbE5XWUpkal9fel9yX2E2M0Z6MGZtN0JJcjRwNVFxZVlwdHFDeGVYelgycFNWelp2RXRDOENVZHV2T0hJY19oeURWZDJ3UzRLQWxr
QURTSl9pMi1MOUFtTXhTRzVfcWItWXpWcHctcllsVi0zUW5DMG8xWm95WWNUd3E0VlFNTXhrZ09TalZyc2VYazlfMDJabmtYOEsxaDJuQ1UzRkZ0a3E1LUtSV0NIM2tmRFdnb2ZTMUdPaHM0SjhTRXhqM1hjNnpCZnZJdXRPaXdHZnB4UUJjU2FKb2RZUW5C
QURTSl9pMlBxR05fMGNDRkI4Zmtsa0tiMUo

QURTSl9pMDlhSDRJQlRpQzlYRUlGbzBtMDc5LV9Ta1ZydWxlOTFfOUdHWUhNY3J1bDJ6S0xEdlJkNktFdy1mbGpEcFhWVk1JcGNlVkVyem1ZRnpIcmFreld1dDBIMkdQd0dyUlRmNlRPSWJtbUhtdVhZN3lwM3ZZQ2R5SnRvMkRpRGtqQ0JUVkduYTRkSm92Y2xj
QURTSl9pMHNnbk1JdkxqaHVvZUVJdlNDcXJTeHBuMFgzZi1KREdvTGZvREtoR3dDYnFqdmI3VEdsRzhaM0Zld1VvYjdhelRlbkVrMnI2X3NSRTRKNHlKUkVjWFRGMGZtOTA3U1Q3QVpTWTBtUEl1ZTJlQV9menA2NGM3Z29uZ1NITE81ZDNuNDY2Q3M1azRLTW5V
QURTSl9pMzRYU1pWMWM0Unl6Y3F0WjhsM0tlX0FFR1ZVZGlid1l3bDRoU0s5dzFSaW1qalMtZ0dRQnl3cjVRczM1VUIwUTlCalhfZW50QUxTMkIyamxLdkk0X3lQOXRwTlhnVUthLUJZY0s3TDVxVVhYWk1QaTBWRlY4ZWdMZ3lVY2JZRWd1TDFUUEp4UGZxaHV3
QURTSl9pMEN1QnJQUVFaVjVEU0FEV2U1VGdqamtQQV8xcmtCbUpobzlwM2pUS1FaTElzRllmMEduYk1sVHpMS3dQRnh2aFFrRUV1cWdPcHdybWx4WW5yb01GdUZEbGh0Rk5jLVlQeWxFTm11cW53RXIwcmR0dTA2ZF9JeUlBS3pQWG5uUXdNb3RDVFcxYm44Rk5v
QURTSl9pMlphVWF3ZmEzMmtWSngwc2NIb2tvNVljVTlBRi1JYWR5aDF2TWRwbHBIQ3JWWENxcUg3dVpzRUlGWVF5QTNCeEdHQnNDMGVkQnNscmU5dTBkbmI0SnBaa0pMaFZOMjRCZV9DUXo3Q21OLWZvV3RQRmRpN0RPbl9Ec0ZBNWNGRHhYOGFtNDFOcnFnWDdn
QURTSl9pMEhkN1N

...

QURTSl9pMXJsZ0VwVmVIa1dqckVFaDc5RVVjM3hKb0h6VFk4cXNsTHF2WlFVSndzbTRNTkl5cFEwMzdLLTMyNTdjR2MtOUJIQ0RJYTFZODBRZnZzTEpjc19PdU9sc1JWZEh0UWs4YXFCYWZsNmt0c3dyb1U4d0ZjdTBkMGQxQzNaWkpzV2ljRTdTRTBwTDF1c2lF
QURTSl9pM1cxbTVUeDJBc1laZUVEcXMtQXpGcmM2dnZQdlpUNkF2eDJMSGpWeHo5TWJNQk10R0FnenlIeFlvampFVkx0ZG9tZnN1SE5xRTlNUFNaRmt1YVVkMTc0RDUxOEFRclRjd2ZRVmFNN2hBWHlQMjR5YzZoNGNjTEFKNXlMaGZvRDJ5RTBHRTF4aWN1TW5r
QURTSl9pMlc3YmNPWXJKVWoxNlAycjBBNHZTZ0NNWmtvWjJucVFkVzZDeUNxdUVjaGhXSnROdGc4Ny1TU2RJSVpVV3lwS214TzRlNENlcnZOZHU1a21RZHM4LVBlSnZFcU5iZjRObnZMdS1ONFVqQk43WUFkcklINktvempPYXRDUG1jc25GMGUzN0ZGMFZtUjFv
QURTSl9pMXQ0aU9uY29Gc1BzeklWVWlTQ0UzWDM4UFprZWM5aldlYkNPSW1hdGNiN3dGV0ZKM0FuSmVBVzNBUWdJd04zRGRibHlNYXpVS2t4WkMxXzJUWllmU2stRWdETkpQQ19fR3pXVGdYZHZtWmNQNjMyX1FuV1hkTGpSM3hMY2JJY3JQMzlHbVdUM1JxbFNR
QURTSl9pM3NyUEY3SXlOTEZxZm9OTkZTMUNHXzN0bTRRNk5xNHN6TFhOZ0QzNjQ4WFhUWTFiYzJ2d2FqV2NQZDRLSGFtazJWMjRGVWlZNS14ZV9fbTNITHd0YTAtcTRlOXNYMlZpUWVpcEFtNHBzYjd1V09IVGM1ZnZuc0hGZGJ2T0JzbjFadS1vNllnNXRGcjBB
QURTSl9pMXJVNk9

QURTSl9pM0FhTVlHdy1FMHBaNjg0R01wWm9oYWtJN3oyRTV3Vy1kOXJISE5fa0FwTnlUN3VZc3ZLSkFkazZkUjZ3S2NKTmVQLUgyT2MxTENNVk9SdDZ0cF9KbGVRYUNybk5zYzRRTnpxOF81b3lFWWlERjhYcnUxcGhCMWFOUDlqTjYyTUh3RnZqcG0wdXF0ck5n
QURTSl9pMWtXbms0MUxTNm5PaWFRMkp2cmFTTUlPb3JVeUJPRFVDVFEyS2NfV0VFZXQzZXU4NDdEeHFsRm96OWd5M0kwZGN5YzF2Yi1LQUVkdkRHVGloLU1Tb3VLUzduVEVfOTRjemktTWExNFFxUFlhMGdaXzdPMVE3bzlBSnBKTVNHQU5FR2ZhSEMxNDRDNUY0
QURTSl9pMGN6Z3NuWVlqWC1kMXIyLUdTQUlqVkRoSnZTLVJGQnBjWEZDdFkzWG42Z1JiSkdHbE1ETGl6MllnTENEQ1puWnNQdFZwYUc1dEZIZk1CbGQzSWhJVDR3NHp2YnMydW1iM19maHhnMkNxR1hxbmtUemZEOHNJcC1NLXVuU2V2YzdhZVU2REs0UFZHaXFV
QURTSl9pMTVfZTNZNERtQm1tV0prVWZka3ZlUFd5ZWNXT2hBYTI3T20xdmgyTjJUQndhTzJFWkFFNUZZYUxWVF9IZmhiSk9nZWt1SGVGbE9tTlVUUUd6YXY1cnhPQ1FvQXdpdG5EeHFZY2pQSzVvVHVkMnBBVjBjUmNaVVczaVlUanpKWFFXME5QWDJ1cnQtelZz
QURTSl9pM1k3QVl4ck1ySWtIS21GaTUzclhyejdiSUlPbndSM29mcTR1TnFpWWljd3hFT3BSeTR6M2xsMk1jOHVTQ2tCX1FrNUxVR2lBa2VuS3BVazI5R01hQl9id0o3RVh1RmZNMnJuQV80ZDU2T3dzdkpuRWJaZjVCZWR4cU42eV9ZdmdZNU04RHp5MGdzMkt3
QURTSl9pMmF5Mjh

QURTSl9pM2xuQnptdnZKUkNiemx3V3VfMnZid21xYXJrb29hMHU5UXM1OGdxWl90MUs5c1pRR3RGcVp4Y3ZiTnQtdVdmSXBjZ24yTXNuZDVrWlVJc2syVEZlS202M1IwM1dzT3hkSFNQemQ0LW1ILW4ySElDZVdxcUMtVVFaRXRwT3I1My1JNDNTaFJVa19CZXFV
QURTSl9pMnhQc3ZzLVdOLUJtRXZhNUt6dnh2dU1INHdycWVUc25TT3p2cV9zODhIQzgzNzFyQWtCTVNocW5lMEY3NTFQb1g1UDBFQ3lRQm9acl9ocGdOby00MjRRdXNmSHI3VFUwVkJiLVZSbnFycFlKU3RHOUYwd3ltaDZjYUt6a1J1Wk9raHlhdEd3TjRTNjFN
QURTSl9pMlBscUhFNU5XUTVhUUtaNjVfZHJDVHVIcG45aGlFN29yQWNnUWcyVlV6YzZ6YmtkREk0SmhLUzNScGgzaGhXMUdTN2I2cGM4Z1JkRXBmZmJmY21lUjlhRVUwdFdsOXpwSXI0VVRwMXNreDYxRThKQWN2ajBnR29Ya204eDU2YWJnVkRzTUc1QW1wbHE4
QURTSl9pMG0xQ1Z5MEJxT1A1X0FCb1M5SXgyeHc3UDV3UTRDTVdvX2RvMjVyTWl6ZFV4MFl5X2J0TnNPTWo2bHBXZjZ4ZlVoV053Y1o0clVXQ2NCY1FCaVJETmZhSEVFTU9LYUpLTHoxWHB1cTREMzZ4WmRoUThJVzJPUW1Mc0V3b2dUemQtaFFNSHBON3c4SDFR
QURTSl9pMzNUU2Joc0dSQ1JKTkZicG5WSDk2VG5OOVVYZXZjSmRNUnN0Z2VXVk43WU1ZdThRQUV5TWx2RnBUdVZqYWx0M0RJc0V3R1AwSXhnSlhrWmRBQ21INXJ6dzN3eEQ1ZFQ1SjZmbG5aRzBHSm1jQUdhZFJEWFV6Q192MVc3LUhsZ1gzR2h3bWhOcVBoeW5B
QURTSl9pM0xTRFF

QURTSl9pMkhNVmd2T3pCeHYyejJhRjJFZXNPMlgxWmtEYkNGNjI5SnJfelN2bzdXcUs2YmRSRnllbGZjTW9VdTJlUkJEMlVXU0I3NDFqVnZoVDdBRG5KRlo2bURlcFZvdkQ5NGt1NERkS2xJcWFkNjJ1bEdpSWZidmpWZVl6T21adF9oeklZQkstWmJkdTlCeWVz
QURTSl9pMGlOc01LTTZLQTlOX0NsVGpJeEZxQnlEZHloZVdKMHdhUDdWQUpnLUl4dUdOeEQyRXd5anFGN2JSVlA3M0FBT29WRjJmS1duVEhodDlPa2tDTGcyd3l4MURtb1Z0SUg0T0xCM3hzZEt0RzlIQndza1p0SDF3R2xyalRDYXlVcXJWZERNVjVyREZLMU13
QURTSl9pMUxteWsxb01IU3FRUmNHOUZQVUVWY1BZSFdDREVfSVkyZ1laNnZ6X1doazVtXzJYOTFPWnpQY1pQcUhYbnNJdVgyRGtPc1ZXdzdfam9qMGh0WXp6RExVMnhvVnk0MFVMR1ZOYmNVWVgxbkpsNTk1cnZodHBWcC1RNHRKVlJBcDNWTHd4U3FHamhNZ1hV
QURTSl9pMVFfcUpMdmUwRUxGUGtVc1hzTlVLMEJHTjFoR05fN0FoLVE1bnhTTzNNVHJCQ1RnY3kycUFWZHliVnNsR2Y1cVNPNW9PRXA3S3I1ekxmODdwRkRqRGNzUlZodF9qTGFXWnpFUmRLOTVjSDI2U2VtU0lyYlp4b1ZkeV9WYUZIVXVuWTlLOG0yVlBQTTJz
QURTSl9pMkxIcVhmMzVsV2xKVExFNnBQLThjdVZ0MksxS294T1I1VDNzUlJiR2U5LTlxaDlVNnMwX0NmNW9MbHlHakV4VzEyQ3l6Um03bU05NDlBd1pXVnlpTXIwLVY1Z2wySE4yZENfVEZ6UEd3SGowUXNpOEVRRE1oM3ZhZ01rWkNFbW8xSnlpX1VTb0hFWHRB
QURTSl9pM2o0SHN

QURTSl9pMFRvQnhGSGxhcldLTEhxNlNWRU1Nb2xlMWp3R1N1NGVoX09QeTNSM2dHLXpsdUZZSTRiLTB6UWt5Tkx3OWctdlEyakcxNlByUWczMTVFVUlMUm5lVl9CRGJLeHdkWGtCQlc0NWVlYTJueEpxVFdTS0liVUhXU3VhS0tVTXNEbDBGMXI3Nk1VYTZ4UGZv
QURTSl9pMzdjNFFWSDdROVMzMFI1VjIyNjBwTnUxc0ltanNaem1fY1Nkb2htQmY5UlBSVUFpQjRrVzNlb2otU1RRQ3dLNGlfemVMOTVZemhwT1BteGZmWnJKeTluV2NsWUJDcXJlQ3psX3NudGtxRDM5dnJ0aVBqVm02OXFOYXBZQldCNWY2SHlNUTVVRndnWFVZ
QURTSl9pMGg1UXFfY2dFeU9GTUw2UzVXM19USnhRa1pUb3MxQnNqSUFNLVRuUXpqbHlhUWNwdEJzLXBfcHZHejllLXhyaXhNV0c1eFZzeXVRZFlIamNuQXBGRmR2QTFxeTk1SkdTb0lxbXdJem5CY1d3LVJ1UGFVYzBKbnAwM0VFRHhTRGRPQjBBbkZyd3pxWHVV
QURTSl9pMGVabExTQ0hwOHBGXzhpV29LS29rT3h6SFdUbGdWZGc3dU5MaDNrS3BtdFRBd0lsbU0wWU5hOEdrOEJIc291eUxhNXZDa0o2dm41YlNWQjJYT0dkWWxINXJJY1hCSXlweFRfU202eGNBaXpxME8xdUNDWi1fbUhJaGZGUnB3SUxNU2s4b3BDNk9KRmJ3
QURTSl9pMnJiT0Z3d2Z3d214dVB0YzAxRm9sSldqZTNxbGZaS05xZ0cxMzRDeGMwTGpSYTI5SEk0dFhnSVhxUlFiaGNJNC1oLUV6ZU03a2hFWTJFdW9IbHpEa2NnV1l0el9kUmh5MHdPWl9kdmVBRGduYzlYd296MXJCSm1UY29JejVzV0VwX3Q3dlJqLWVjU0dF
               